In [229]:
# Adding neccessary imports
import pandas as pd
import numpy as np
import pandas as pd

# Function to display all columns in dataset when viewing in notebook
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.expand_frame_repr', False)  # Don't wrap to multiple lines


In [230]:
# Load dataset
df = pd.read_csv("modelcraftdata.csv")

In [231]:
df.head(2)

,Timestamp,What is your gender,Year of graduation,Title of course studied,Polytechnic/University attended,What is your highest level of education?,What best describes your current status?,How many jobs have you had since graduation including your current one?,Have you completed your NYSC?,"If you answered yes/ongoing to the previous question, what year did you (or will you) complete your NYSC?",Did you get your first full time job through your NYSC placement?,"Thinking about your first employment after graduation, what was your job level?",What is/was your job role?,"What sector/industry is your company in? (E.g. Banking, Agriculture, Telecommunication)","Still thinking about your first employment, what is/was your monthly income level?","Did you need your higher education qualification to get your first job (the actual qualification, not the subject of study)?",What was the most significant reason for deciding to take your first job?,"Thinking about your current employment, what is your job level?",What is your job role?,"What sector/industry is your company in? (E.g. Banking, Agriculture, Telecommunication).1",What is your current monthly income level?,"Did you need your higher education qualification to get this job (the actual qualification, not the subject of study)?",What was the most significant reason for deciding to take this job?,Which employer in the country do you think offers the best opportunities for graduates?,Reason why?,Which sector is your most preferred sector to work in?,What currency are you currently paid in?,Approximately how many hours a day do you currently work?,"As far as you are aware, what was most important to your current employer about your qualification?",How do you find out about your job(s)?,"Thinking about your current employment, did you work for your employer before or during your higher education study?",Which form of transport do you use the most?,Were you able to rent an apartment or buy a car from the salary you got from your first job?,My course of study prepared me well for employment,My course of study prepared me well for further studies,Which of these skills/knowledge did your higher education prepare you for?
0,2/8/2018 9:05:23,Male,2013,Electrical & Electronic Engineering,"Obafemi Awolowo University,Ile-Ife",Bachelor's degree,Working full time (paid employment),3,NaN,NaN,NaN,Experience,NaN,Technology,"‎₦50,000 - ‎₦99,999",No: the qualification was not required,It fitted into my career plan/it was exactly t...,Managerial,NaN,Technology,"‎₦200,000 - ‎₦249,999",NaN,It fitted into my career plan/it was exactly t...,NNPC,NaN,Engineering,Naira,8.0,No one thing was important,Internship,No,Bus,Yes: From my first job,No,No,"Critical thinking skills, Ability to solve com..."
1,2/15/2018 12:11:32,Male,2014,Psychology,"Obafemi Awolowo University,Ile-Ife",Bachelor's degree,Working full time (paid employment),2,Yes,2017.0,No,Entry level,NaN,Media,"‎₦20,000 - ‎₦49,999",Yes: the qualification was a formal requirement,To see if I would like the type of work it inv...,Experience,Option 1,Media,"‎₦100,000 - ‎₦149,999",Yes: the qualification was a formal requirement,It fitted into my career plan/it was exactly t...,Andela,Impression formed from the media.,Media,Naira,8.0,IT/SIWES/Internship experience (gained as part...,Internship,No,BRT,No: From my current job,Strongly Disagree,Agree,"Critical thinking skills, Ability to solve com..."


In [232]:
# Drop timestamp column
df.drop("Timestamp", axis=1, inplace=True)

In [233]:
df["missing_value_column"] = df.isna().sum(axis=1)

In [234]:
condition = df["missing_value_column"] >= 15
df = df[~condition]
len(df)

3432

In [235]:
# Function to report number of missing values
def stats():
    return df.isna().sum()

In [236]:
# Show the number of missing values per column
stats()

What is your gender                                                                                                                0
Year of graduation                                                                                                                 0
Title of course studied                                                                                                            0
Polytechnic/University attended                                                                                                    0
What is your highest level of education?                                                                                           0
What best describes your current status?                                                                                           0
How many jobs have you had since graduation including your current one?                                                            0
Have you completed your NYSC?                                        

In [237]:
# Gender values 
df[df.columns[0]].value_counts()

What is your gender
Male                 1772
Female               1659
Prefer not to say       1
Name: count, dtype: int64

In [238]:
# Function to group "prefer_not_to_say" gender rows with then average row they are most simlar to 

def jaccard_similarity(set1, set2):
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    return intersection / union

# Iterate through 'prefer not to say' rows
for index, row in df[df['What is your gender'] == 'Prefer not to say'].iterrows():
    prefer_not_to_say_set = set(row.values[1:])  # Assuming values are categorical
    
    male_set = set(df[df['What is your gender'] == 'Male'].values[0][1:])
    female_set = set(df[df['What is your gender'] == 'Female'].values[0][1:])
    
    male_similarity = jaccard_similarity(prefer_not_to_say_set, male_set)
    female_similarity = jaccard_similarity(prefer_not_to_say_set, female_set)
    
    if male_similarity > female_similarity:
        df.at[index, 'What is your gender'] = 'Male'
    else:
        df.at[index, 'What is your gender'] = 'Female'


In [239]:
# Verification
df[df.columns[0]].value_counts()

What is your gender
Male      1772
Female    1660
Name: count, dtype: int64

In [240]:
df[df.columns[1]].value_counts()

Year of graduation
2015    935
2016    745
2017    646
2014    632
2013    474
Name: count, dtype: int64

In [241]:
for i in [df[df.columns[2]].value_counts()]:
    print(i)

Title of course studied
Computer Science                                        331
Accountancy                                             257
Economics                                               224
Mass Communication (Communication and Language Arts)    146
Electrical & Electronic Engineering                     138
                                                       ... 
Medical Radiography and Radiological Sciences             1
Music                                                     1
Child Dental Health                                       1
Archeology and Tourism                                    1
Social Sciences Education                                 1
Name: count, Length: 125, dtype: int64


In [242]:
for i in df[df.columns[3]].unique().tolist():
    print(i)

Obafemi Awolowo University,Ile-Ife
Bells University of Technology, Otta
Ekiti State University
University of Benin
Bowen University, Iwo
Covenant University Ota
University of Ibadan
University of Lagos
Babcock University,Ilishan-Remo
Yaba College of Technology, Yaba, Lagos State.
University of Port-Harcourt
Lagos State University Ojo, Lagos.
Federal University of Technology, Minna.
Ladoke Akintola University of Technology, Ogbomoso
Federal University of Technology, Owerri
Caleb University, Lagos
Adekunle Ajasin University, Akungba.
University of Ilorin
Federal University of Technology, Akure
Federal Polytechnic Kaura Namoda, Zamfara State.
University of Nigeria, Nsukka
Madonna University, Okija
Modibbo Adama University of Technology, Yola
Ambrose Alli University, Ekpoma,
University of Maiduguri
National Open University of Nigeria, Lagos.
Abia State University, Uturu.
University of Uyo
Crawford University Igbesa
Olabisi Onabanjo University Ago-Iwoye
Anambra State University of Science &

In [215]:
df[df.columns[4]].value_counts()

What is your highest level of education?
Bachelor's degree                  2892
Master's degree                     239
Higher National Diploma (HND)       222
Ordinary National Diploma (OND)      55
MBA degree                           18
PhDs/Doctorate Degree                 6
Name: count, dtype: int64

In [243]:
df[df.columns[5]].value_counts()

What best describes your current status?
Working full time (paid employment)                                                           1362
Youth Corper (NYSC)                                                                            527
Unemployed                                                                                     445
Self-employed/freelance/entrepreneur                                                           420
Voluntary or other unpaid work or on an internship                                             324
Due to start a job in the next month/developing a professional portfolio/creative practice     101
Engaged in full-time further study, training or research                                        85
Taking time out to prepare for further studies, GMAT, professional exams                        77
Engaged in part-time further study, training or research                                        49
Doing something else (e.g. looking after home or family)            

In [244]:
df[df.columns[6]].value_counts()

How many jobs have you had since graduation including your current one?
1     1776
2     1052
3      456
4      101
5       30
10       6
6        6
7        4
8        1
Name: count, dtype: int64

In [245]:
df[df.columns[7]].value_counts()

Have you completed your NYSC?
Yes        2423
Ongoing     517
No          491
Name: count, dtype: int64

In [246]:
df["Have you completed your NYSC?"].fillna("No", inplace=True)

In [247]:
# Verification
df[df.columns[7]].value_counts()

Have you completed your NYSC?
Yes        2423
Ongoing     517
No          492
Name: count, dtype: int64

In [248]:
stats()

What is your gender                                                                                                                0
Year of graduation                                                                                                                 0
Title of course studied                                                                                                            0
Polytechnic/University attended                                                                                                    0
What is your highest level of education?                                                                                           0
What best describes your current status?                                                                                           0
How many jobs have you had since graduation including your current one?                                                            0
Have you completed your NYSC?                                        

In [249]:
df["If you answered yes/ongoing to the previous question, what year did you (or will you) complete your NYSC?"].fillna("Not Applicable", inplace=True)

In [250]:
# Verification
df[df.columns[8]].value_counts()

If you answered yes/ongoing to the previous question, what year did you (or will you) complete your NYSC?
2017.0            985
2016.0            590
2018.0            544
Not Applicable    493
2015.0            415
2014.0            229
2019.0            108
2013.0             68
Name: count, dtype: int64

In [252]:
df[df.columns[9]].fillna("No", inplace=True)

In [253]:
stats()

What is your gender                                                                                                                0
Year of graduation                                                                                                                 0
Title of course studied                                                                                                            0
Polytechnic/University attended                                                                                                    0
What is your highest level of education?                                                                                           0
What best describes your current status?                                                                                           0
How many jobs have you had since graduation including your current one?                                                            0
Have you completed your NYSC?                                        

In [256]:
no_response_columns = [ 'Thinking about your first employment after graduation, what was your job level?',
                        'What is/was your job role?',
                        'What sector/industry is your company in? (E.g. Banking, Agriculture, Telecommunication)',
                        'Still thinking about your first employment, what is/was your monthly income level?',
                        'Did you need your higher education qualification to get your first job (the actual qualification, not the subject of study)?']

In [259]:
df[no_response_columns].fillna("No Response", inplace=True)

C:\Users\HP\AppData\Local\Temp\ipykernel_11572\137649480.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[no_response_columns].fillna("No Response", inplace=True)


In [260]:
stats()

What is your gender                                                                                                                0
Year of graduation                                                                                                                 0
Title of course studied                                                                                                            0
Polytechnic/University attended                                                                                                    0
What is your highest level of education?                                                                                           0
What best describes your current status?                                                                                           0
How many jobs have you had since graduation including your current one?                                                            0
Have you completed your NYSC?                                        

In [224]:
df["Did you get your first full time job through your NYSC placement?"].replace("missing", "No", inplace=True)

In [225]:
# Verification
df[df.columns[9]].value_counts()

Did you get your first full time job through your NYSC placement?
No                              2371
Yes                              607
I have not completed my NYSC     454
Name: count, dtype: int64

In [226]:
df[df.columns[10]].value_counts()

Thinking about your first employment after graduation, what was your job level?
missing                        1776
Entry level                    1141
Clerical and administrative     256
Experience/Professional         206
Managerial                       49
Executive Director                3
Experience                        1
Name: count, dtype: int64

In [227]:
df[df.columns[11]].value_counts()

What is/was your job role?
missing                                              1778
Teacher                                               210
Customer Service                                      201
Finance/Accounting                                    120
Sales                                                 102
IT Support                                             75
Content Creator/Content Writer/ Content Developer      70
Marketing                                              70
Doctor, Nursing And Other Medical Related Roles        56
Data Entry                                             55
Business Analyst                                       46
Executive Assistance/Personal Assitance                46
Business Developer                                     45
Business Support                                       43
Creatives                                              40
Human Resources                                        38
Researcher                                   

In [228]:
df[df.columns[12]].value_counts()

What sector/industry is your company in? (E.g. Banking, Agriculture, Telecommunication)
missing                            1776
Education                           251
Banking / Financial Services        146
Technology                          137
Consulting                           87
Telecommunications                   85
Healthcare                           82
Construction / Real Estate           78
Advertising / Marketing / Comms      75
Others                               73
Ecommerce/Internet                   55
Media                                55
Engineering                          54
NGO                                  46
Oil & Gas / Mining                   42
Legal                                41
Trade / Services                     39
Hospitality/Leisure                  37
Manufacturing / Production           37
Insurance                            32
Creatives/Art/Design                 32
Retail / Wholesales                  30
Agriculture/Poultry/Fishing     

In [ ]:
df["Thinking about your first employment after graduation, what was your job level?"].replace("missing", "Not Applicable", inplace=True)

In [23]:
df["If you answered yes/ongoing to the previous question, what year did you (or will you) complete your NYSC?"].value_counts()

If you answered yes/ongoing to the previous question, what year did you (or will you) complete your NYSC?
2017.0    1513
2018.0     832
2016.0     790
2015.0     541
2014.0     303
2019.0     224
2013.0      75
Name: count, dtype: int64

In [24]:
df["If you answered yes/ongoing to the previous question, what year did you (or will you) complete your NYSC?"].fillna("Not Applicable")

0       Not Applicable
1               2017.0
2       Not Applicable
3       Not Applicable
4               2016.0
             ...      
5214            2016.0
5215    Not Applicable
5216            2015.0
5217            2017.0
5218            2018.0
Name: If you answered yes/ongoing to the previous question, what year did you (or will you) complete your NYSC?, Length: 5219, dtype: object

In [25]:
df["If you answered yes/ongoing to the previous question, what year did you (or will you) complete your NYSC?"].fillna("Not Applicable", inplace=True)

In [26]:
stats()

What is your gender                                                                                                                0
Year of graduation                                                                                                                 0
Title of course studied                                                                                                            0
Polytechnic/University attended                                                                                                    0
What is your highest level of education?                                                                                           0
What best describes your current status?                                                                                           0
How many jobs have you had since graduation including your current one?                                                            0
Have you completed your NYSC?                                        

In [27]:
df[df["Did you get your first full time job through your NYSC placement?"].isna() == True]

,What is your gender,Year of graduation,Title of course studied,Polytechnic/University attended,What is your highest level of education?,What best describes your current status?,How many jobs have you had since graduation including your current one?,Have you completed your NYSC?,"If you answered yes/ongoing to the previous question, what year did you (or will you) complete your NYSC?",Did you get your first full time job through your NYSC placement?,"Thinking about your first employment after graduation, what was your job level?",What is/was your job role?,"What sector/industry is your company in? (E.g. Banking, Agriculture, Telecommunication)","Still thinking about your first employment, what is/was your monthly income level?","Did you need your higher education qualification to get your first job (the actual qualification, not the subject of study)?",What was the most significant reason for deciding to take your first job?,"Thinking about your current employment, what is your job level?",What is your job role?,"What sector/industry is your company in? (E.g. Banking, Agriculture, Telecommunication).1",What is your current monthly income level?,"Did you need your higher education qualification to get this job (the actual qualification, not the subject of study)?",What was the most significant reason for deciding to take this job?,Which employer in the country do you think offers the best opportunities for graduates?,Reason why?,Which sector is your most preferred sector to work in?,What currency are you currently paid in?,Approximately how many hours a day do you currently work?,"As far as you are aware, what was most important to your current employer about your qualification?",How do you find out about your job(s)?,"Thinking about your current employment, did you work for your employer before or during your higher education study?",Which form of transport do you use the most?,Were you able to rent an apartment or buy a car from the salary you got from your first job?,My course of study prepared me well for employment,My course of study prepared me well for further studies,Which of these skills/knowledge did your higher education prepare you for?
0,Male,2013,Electrical & Electronic Engineering,"Obafemi Awolowo University,Ile-Ife",Bachelor's degree,Working full time (paid employment),3,Not Applicable,Not Applicable,NaN,Experience,NaN,Technology,"‎₦50,000 - ‎₦99,999",No: the qualification was not required,It fitted into my career plan/it was exactly t...,Managerial,NaN,Technology,"‎₦200,000 - ‎₦249,999",NaN,It fitted into my career plan/it was exactly t...,NNPC,NaN,Engineering,Naira,8.0,No one thing was important,Internship,No,Bus,Yes: From my first job,No,No,"Critical thinking skills, Ability to solve com..."


In [56]:
not_applicable_columns = ["Thinking about your first employment after graduation, what was your job level?", 
                          "What is/was your job role?",
                          "What sector/industry is your company in? (E.g. Banking, Agriculture, Telecommunication)", 
                          "Still thinking about your first employment, what is/was your monthly income level?",
                          "Did you need your higher education qualification to get your first job (the actual qualification, not the subject of study)?",
                          "What was the most significant reason for deciding to take your first job?"]                                                                                                                                             

In [62]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy="constant", fill_value="Not Applicable")

df[not_applicable_columns] = imputer.fit_transform(df[not_applicable_columns])



In [63]:
stats()

What is your gender                                                                                                                0
Year of graduation                                                                                                                 0
Title of course studied                                                                                                            0
Polytechnic/University attended                                                                                                    0
What is your highest level of education?                                                                                           0
What best describes your current status?                                                                                           0
How many jobs have you had since graduation including your current one?                                                            0
Have you completed your NYSC?                                        

In [66]:
next_set = ['Thinking about your current employment, what is your job level?',
       'What is your job role?',
       'What sector/industry is your company in? (E.g. Banking, Agriculture, Telecommunication).1',
       'What is your current monthly income level?',
       'Did you need your higher education qualification to get this job (the actual qualification, not the subject of study)?',
       'What was the most significant reason for deciding to take this job?',
       'Which employer in the country do you think offers the best opportunities for graduates?',
       'Reason why?', 'Which sector is your most preferred sector to work in?',
       'What currency are you currently paid in?',
       'Approximately how many hours a day do you currently work?',
       'As far as you are aware, what was most important to your current employer about your qualification?',
       'How do you find out about your job(s)?',
       'Thinking about your current employment, did you work for your employer before or during your  higher education study?',
       'Which form of transport do you use the most?',
       'Were you able to rent an apartment or buy a car from the salary you got from your first job?',
       'My course of study prepared me well for employment',
       'My course of study prepared me well for further studies',
       'Which of these skills/knowledge did your higher education prepare you for?']

df[next_set] = imputer.fit_transform(df[next_set])

In [67]:
stats()

What is your gender                                                                                                             0
Year of graduation                                                                                                              0
Title of course studied                                                                                                         0
Polytechnic/University attended                                                                                                 0
What is your highest level of education?                                                                                        0
What best describes your current status?                                                                                        0
How many jobs have you had since graduation including your current one?                                                         0
Have you completed your NYSC?                                                             

In [69]:
df["Did you get your first full time job through your NYSC placement?"].fillna("Not Applicable", inplace=True)

In [70]:
stats()

What is your gender                                                                                                             0
Year of graduation                                                                                                              0
Title of course studied                                                                                                         0
Polytechnic/University attended                                                                                                 0
What is your highest level of education?                                                                                        0
What best describes your current status?                                                                                        0
How many jobs have you had since graduation including your current one?                                                         0
Have you completed your NYSC?                                                             

In [75]:
df

,What is your gender,Year of graduation,Title of course studied,Polytechnic/University attended,What is your highest level of education?,What best describes your current status?,How many jobs have you had since graduation including your current one?,Have you completed your NYSC?,"If you answered yes/ongoing to the previous question, what year did you (or will you) complete your NYSC?",Did you get your first full time job through your NYSC placement?,"Thinking about your first employment after graduation, what was your job level?",What is/was your job role?,"What sector/industry is your company in? (E.g. Banking, Agriculture, Telecommunication)","Still thinking about your first employment, what is/was your monthly income level?","Did you need your higher education qualification to get your first job (the actual qualification, not the subject of study)?",What was the most significant reason for deciding to take your first job?,"Thinking about your current employment, what is your job level?",What is your job role?,"What sector/industry is your company in? (E.g. Banking, Agriculture, Telecommunication).1",What is your current monthly income level?,"Did you need your higher education qualification to get this job (the actual qualification, not the subject of study)?",What was the most significant reason for deciding to take this job?,Which employer in the country do you think offers the best opportunities for graduates?,Reason why?,Which sector is your most preferred sector to work in?,What currency are you currently paid in?,Approximately how many hours a day do you currently work?,"As far as you are aware, what was most important to your current employer about your qualification?",How do you find out about your job(s)?,"Thinking about your current employment, did you work for your employer before or during your higher education study?",Which form of transport do you use the most?,Were you able to rent an apartment or buy a car from the salary you got from your first job?,My course of study prepared me well for employment,My course of study prepared me well for further studies,Which of these skills/knowledge did your higher education prepare you for?
0,Male,2013,Electrical & Electronic Engineering,"Obafemi Awolowo University,Ile-Ife",Bachelor's degree,Working full time (paid employment),3,Not Applicable,Not Applicable,Not Applicable,Experience,Not Applicable,Technology,"‎₦50,000 - ‎₦99,999",No: the qualification was not required,It fitted into my career plan/it was exactly t...,Managerial,Not Applicable,Technology,"‎₦200,000 - ‎₦249,999",Not Applicable,It fitted into my career plan/it was exactly t...,NNPC,Not Applicable,Engineering,Naira,8.0,No one thing was important,Internship,No,Bus,Yes: From my first job,No,No,"Critical thinking skills, Ability to solve com..."
1,Male,2014,Psychology,"Obafemi Awolowo University,Ile-Ife",Bachelor's degree,Working full time (paid employment),2,Yes,2017.0,No,Entry level,Not Applicable,Media,"‎₦20,000 - ‎₦49,999",Yes: the qualification was a formal requirement,To see if I would like the type of work it inv...,Experience,Option 1,Media,"‎₦100,000 - ‎₦149,999",Yes: the qualification was a formal requirement,It fitted into my career plan/it was exactly t...,Andela,Impression formed from the media.,Media,Naira,8.0,IT/SIWES/Internship experience (gained as part...,Internship,No,BRT,No: From my current job,Strongly Disagree,Agree,"Critical thinking skills, Ability to solve com..."
2,Female,2013,Economics,"Bells University of Technology, Otta",Bachelor's degree,Working full time (paid employment),2,Yes,Not Applicable,No,Entry level,Option 1,Consulting,"‎₦100,000 - ‎₦149,999",Yes: the qualification was a formal requirement,It fitted into my career plan/it was exactly t...,Experience,Option 1,Consulting,"‎₦200,000 - ‎₦249,999",Yes: the qualification was a formal requirement,To gain and broaden my experience in order to ...,It depends. However for a business/finance rel...,Impression formed from talking t

In [81]:
pd.DataFrame(df[df.columns].value_counts())

count
What is your gender Year of graduation Title of course studied                            Polytechnic/University attended        What is your highest level of education? What best describes your current status?           How many jobs have you had since graduation inc... Have you completed your NYSC? If you answered yes/ongoing to the previous que... Did you get your first full time job through yo... Thinking about your first employment after grad... What is/was your job role? What sector/industry is your company in? (E.g. ... Still thinking about your first employment, wha... Did you need your higher education qualificatio... What was the most significant reason for decidi... Thinking about your current employment, what is... What is your job role?                          What sector/industry is your company in? (E.g. ... What is your current monthly income level? Did you need your higher education qualificatio... What was the most significant reason for decidi... Which employer in the country do you think offe... Reason why?                                        Which sector is your most preferred sector to w... What currency are you currently paid in? Approximately how many hours a day do you curre... As far as you are aware, what was most importan... How do you find out about your job(s)?        Thinking about your current employment, did you... Which form of transport do you use the most? Were you able to rent an apartment or buy a car... My course of study prepared me well for employment My course of study prepared me well for further... Which of these skills/knowledge did your higher...       
Female              2014               Science Education                                  Auchi Polytechnic, Auchi, Edo State.   Ordinary National Diploma (OND)          Unemployed                                         0                                                  No                            2015.0                                             No                                                 Not Applicable                                     Not Applicable             Not Applicable                                     Not Applicable                                     Not Applicable                                     Not Applicable                                     Not Applicable                                     Not Applicable                                  Not Applicable                                     Not Applicable                             Not Applicable                                     Not Applicable                                     Not Applicable                                     Not Applicable                                     Not Applicable                                     Not Applicable                           Not Applicable                                     Not Applicable                                     Not Applicable                                Not Applicable                                     Not Applicable                               Not Applicable                                     Not Applicable                                     Not Applicable                                     Not Applicable                                          6
                    2013               Science Education                                  Auchi Polytechnic, Auchi, Edo State.   Ordinary National Diploma (OND)          Unemployed                                         0                                                  No                            2014.0                                             No                                                 Not Applicable                                     Not Applicable             Not Applicable                                     Not Applicable                                     Not Applicable                                     Not Applicable                                     Not Applicable                                  

In [3]:
filtered = pd.read_csv("group_data_cleaning.csv")
filtered

,What is your gender,Year of graduation,Title of course studied,Polytechnic/University attended,What is your highest level of education?,What best describes your current status?,How many jobs have you had since graduation including your current one?,Have you completed your NYSC?,"If you answered yes/ongoing to the previous question, what year did you (or will you) complete your NYSC?",Did you get your first full time job through your NYSC placement?,"Thinking about your first employment after graduation, what was your job level?",What is/was your job role?,"What sector/industry is your company in? (E.g. Banking, Agriculture, Telecommunication)","Still thinking about your first employment, what is/was your monthly income level?","Did you need your higher education qualification to get your first job (the actual qualification, not the subject of study)?",What was the most significant reason for deciding to take your first job?,"Thinking about your current employment, what is your job level?",What is your job role?,"What sector/industry is your company in? (E.g. Banking, Agriculture, Telecommunication).1",What is your current monthly income level?,"Did you need your higher education qualification to get this job (the actual qualification, not the subject of study)?",What was the most significant reason for deciding to take this job?,Which employer in the country do you think offers the best opportunities for graduates?,Reason why?,Which sector is your most preferred sector to work in?,What currency are you currently paid in?,Approximately how many hours a day do you currently work?,"As far as you are aware, what was most important to your current employer about your qualification?",How do you find out about your job(s)?,"Thinking about your current employment, did you work for your employer before or during your higher education study?",Which form of transport do you use the most?,Were you able to rent an apartment or buy a car from the salary you got from your first job?,My course of study prepared me well for employment,My course of study prepared me well for further studies,Which of these skills/knowledge did your higher education prepare you for?
0,Female,2013,Economics,"Bells University of Technology, Otta",Bachelor's degree,Working full time (paid employment),2,Yes,NaN,No,Entry level,Option 1,Consulting,"100,000 - 149,999",Yes: the qualification was a formal requirement,It fitted into my career plan/it was exactly t...,Experience,Option 1,Consulting,"200,000 - 249,999",Yes: the qualification was a formal requirement,To gain and broaden my experience in order to ...,It depends. However for a business/finance rel...,Impression formed from talking to people who h...,Consulting,Naira,14.0,The subject(s) I studied,Employer's website,No,Uber/Taxify/Taxi services,No: From my current job,Disagree,Agree,"Ability to work with others, Written communica..."
1,Female,2013,Statistics,Ekiti State University,Bachelor's degree,Working full time (paid employment),3,Yes,2016.0,No,Managerial,Operations Manager,Retail / Wholesales,"50,000 - 99,999",Yes: while the qualification was not a formal ...,To gain and broaden my experience in order to ...,Entry level,Business Analyst,Advertising / Marketing / Comms,"100,000 - 149,999",Yes: the qualification was a formal requirement,It fitted into my career plan/it was exactly t...,Communications Company,Others,Advertising / Marketing / Comms,Naira,8.0,IT/SIWES/Internship experience (gained as part...,"Personal contacts, including family and friends",No,Commercial Vehicles,No,Agree,Agree,Critical thinking skills
2,Male,2015,Electrical & Electronic Engineering,University of Benin,Bachelor's degree,Working full time (paid employment),2,No,2013.0,No,Entry level,Creatives,Advertising / Marketing / Comms,"20,000 - 49,999",No: the qualification was not required,It was in the right location,Experience,Creatives,Creatives/Art/Design,"200,000 - 249,999",No: the qualification was not required,It was in the right l

In [4]:
filtered.isna().sum()

What is your gender                                                                                                                0
Year of graduation                                                                                                                 0
Title of course studied                                                                                                            0
Polytechnic/University attended                                                                                                    0
What is your highest level of education?                                                                                           0
What best describes your current status?                                                                                           0
How many jobs have you had since graduation including your current one?                                                            0
Have you completed your NYSC?                                        

In [9]:
filtered["What is your job role?"].value_counts()

What is your job role?
Customer Service                                     342
Teacher                                              306
Finance/Accounting                                   259
Sales                                                145
Marketing                                            138
IT Support                                           135
Data Entry                                           104
Business Developer                                    99
Content Creator/Content Writer/ Content Developer     93
Human Resources                                       86
Executive Assistance/Personal Assitance               85
Operations Manager                                    85
Doctor, Nursing And Other Medical Related Roles       82
Business Analyst                                      82
Researcher                                            74
Social Media/Digital Marketing                        67
Business Support                                      62
Creative

In [10]:
filtered.shape

(2961, 35)

In [13]:
df.shape

(5219, 36)